In [4]:
import pandas as pd
df = pd.read_excel('../database/PreFinalDataset.xlsx',engine='openpyxl')  

In [5]:
df.head(4)

,tags,headline,url,clean_body,stop_clean_body,target
0,['আমেরিকা দূর পরবাস'],নিউইয়র্কে একুশের গ্রন্থমেলা,https://www.prothomalo.com/life/durporobash/নি...,২০ ফেব্রুয়ারি বিকেল পাঁচটায় জাতিসংঘের সদর দ...,২০ ফেব্রুয়ারি বিকেল পাঁচটায় জাতিসংঘের সদর দপ...,durporobash
1,"['বিবিধ', 'বিনোদন']",abc আজকের আয়োজন,https://www.prothomalo.com/entertainment/abc-আ...,তথ্য ও বিনোদনপ্রেমরোগকথাবন্ধু শারমীন ও লাভ সি...,তথ্য বিনোদনপ্রেমরোগকথাবন্ধু শারমীন লাভ সিক বিপ...,entertainment
2,"['বিবিধ', 'বিশাল বাংলা']",বিজ্ঞানের তিনটি বিষয়ে সম্মান কোর্স চালুর দাবি,https://www.prothomalo.com/bangladesh/বিজ্ঞানে...,কক্সবাজার সরকারি কলেজে পদার্থ রসায়ন ও প্রাণিব...,কক্সবাজার সরকারি কলেজে পদার্থ রসায়ন প্রাণিবিদ্...,bangladesh
3,"['মতামত', 'চিঠিপত্র']",এমআরপি সংশোধন ফি,https://www.prothomalo.com/opinion/letter/এমআর...,বাংলাদেশে ২০০৯ সাল থেকে এমআরপি পাসপোর্ট কার্য...,বাংলাদেশে ২০০৯ সাল এমআরপি পাসপোর্ট কার্যক্রম আ...,letter


In [6]:
####### target variable anaysis ########
len(df)

10000

In [7]:
df.target.value_counts() 

bangladesh       3668
crime            1476
cricket           424
politics          371
business          343
                 ... 
reaction            1
poem                1
banglanewyear       1
recipe              1
interior            1
Name: target, Length: 65, dtype: int64

In [8]:
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib 
font = {'family' : 'Times New Roman',
#         'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

target_df = df.target.value_counts() 
plt.figure(figsize=(20,12))
sns.barplot(target_df.index, target_df.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=18)
plt.xlabel('Target', fontsize=18)
plt.xticks(rotation=90)
plt.savefig('Initial All Target.jpeg')
plt.show()



ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
taken_df = df.target.value_counts().rename_axis('target').reset_index(name='Frequency') 

In [ ]:
taken_df 

In [ ]:
######## find which target comes only one time ###########
taken_df = taken_df[taken_df.Frequency>100]   

In [ ]:
taken_df

In [ ]:
plt.figure(figsize=(20,12))
sns.barplot(taken_df.target.tolist(), taken_df.Frequency.tolist(), alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=18)
plt.xlabel('target', fontsize=18)
plt.xticks(rotation=90)
plt.savefig('Taken target though imbalanced.jpeg')
plt.show()

In [ ]:
from statistics import mean
avg_num_target = mean(taken_df.Frequency.tolist())
print(avg_num_target)
print(len(df[df.target == 'durporobash']))


In [ ]:
### merge with final ##
df2 = df.merge(taken_df, on='target') 
df2

In [ ]:
del df2['Frequency']

In [ ]:
df2.target.value_counts() 

In [ ]:
from matplotlib.gridspec import GridSpec
import numpy as np
font = {'family' : 'Times New Roman',
#         'weight' : 'bold',
        'size'   : 25}

matplotlib.rc('font', **font)

targetCounts = df2['target'].value_counts()
targetLabels  = df2['target'].value_counts().rename_axis('Target').reset_index(name='Frequency')
targetLabels = targetLabels.Target.tolist()
# Make square figures and axes
cmap = plt.get_cmap('coolwarm')
plt.figure(1, figsize=(18,14))
source_pie = plt.pie(targetCounts, labels=targetLabels, autopct='%1.1f%%', shadow=False)
plt.savefig('imbalancedbalancedpie.jpeg')
plt.show()

In [ ]:
for i in taken_df.target: 
    ind = df2.target[df2.target == i].index
    d2f = df2.drop(ind[300:])

In [ ]:
keep_data = int(avg_num_target)
for i in taken_df.target:  # title df2 is used for reduce time complexity  ### Down sampling ###
    if len(df2[df2.target == i])>keep_data:
        indexNames = df2[df2.target== i ].index
        keep_16 = indexNames[keep_data:]
        df2 = df2.drop(keep_16)
    else:
        pass

In [ ]:
master_copy_df = pd.DataFrame()
for i in taken_df.target:  # title df2 is used for reduce time complexity  ### Down sampling ###
    if len(df2[df2.target == i] )<keep_data:     ### up sampling ##
        clone_df = pd.DataFrame()
        indexNames = df2[ df2.target == i ].index
        multi_factor = round(keep_data/len(indexNames)) # already exist same amount of len(indexNames)
        for j in indexNames:
            clone_df = clone_df.append(df2.loc[[j]*multi_factor],ignore_index = True)
        approved_index = keep_data-len(indexNames)
        clone_df = clone_df.iloc[:approved_index]
        frames = [master_copy_df, clone_df]
        master_copy_df = pd.concat(frames)
    else:
        pass

In [ ]:
final_frame = [df2, master_copy_df]
final_df = pd.concat(final_frame)

In [ ]:
from matplotlib.gridspec import GridSpec
import numpy as np
font = {'family' : 'Times New Roman',
#         'weight' : 'bold',
        'size'   : 25} 

matplotlib.rc('font', **font) 

targetCounts = final_df['target'].value_counts()
targetLabels  = final_df['target'].unique()
# Make square figures and axes
plt.figure(1, figsize=(18,14))
cmap = plt.get_cmap('coolwarm')
source_pie = plt.pie(targetCounts, labels=targetLabels, autopct='%1.1f%%', shadow=False, )
plt.savefig('balancedpie.jpeg')
plt.show()

In [ ]:
############### maximum word counter analysis ###############


In [ ]:
final_df = final_df.sample(frac = 1)
# final_df.to_excel('../dataset/FinalTrainAbleDataset.xlsx')